#Connect to drive

In [1]:
!git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi
%cd RecSys_Course_AT_PoliMi
!pip install -r requirements.txt
!python run_compile_all_cython.py

Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1499, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 1499 (delta 113), reused 231 (delta 100), pack-reused 1255
Receiving objects: 100% (1499/1499), 50.98 MiB | 11.80 MiB/s, done.
Resolving deltas: 100% (844/844), done.
/content/RecSys_Course_AT_PoliMi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.0/978.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess t

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


#Import Library

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from typing import Tuple, Callable, Dict, Optional, List
from scipy.stats import loguniform
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.2 MB/s eta 0:00:00


In [4]:
import warnings
import logging
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

#Load Data

In [5]:
%cd /gdrive/MyDrive/RECSYS
data_train = pd.read_csv('data_train.csv')
data_target_users_test = pd.read_csv('data_target_users_test.csv')
%cd RecSys_Course_AT_PoliMi

/gdrive/MyDrive/RECSYS
/gdrive/MyDrive/RECSYS/RecSys_Course_AT_PoliMi


In [6]:
data_train.columns = ["user_id", "item_id", "Interaction"]

In [7]:
print ("The number of interactions is {}".format(len(data_train)))

The number of interactions is 478730


In [8]:
userID_unique = data_train["user_id"].unique()
itemID_unique = data_train["item_id"].unique()

In [9]:
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(data_train)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))

Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024



#Splitting Data

In [10]:
def preprocess_data(interactions: pd.DataFrame):
    unique_users = interactions.user_id.unique()
    unique_items = interactions.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    interactions = pd.merge(left=interactions,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    interactions = pd.merge(left=interactions,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return interactions


In [11]:
interactions = preprocess_data(data_train)

12638 1 13024
22222 1 22347


In [12]:
interactions

,user_id,item_id,Interaction,mapped_user_id,mapped_item_id
0,1,7,1.0,0,0
1,2,7,1.0,1,0
2,26,7,1.0,24,0
3,36,7,1.0,34,0
4,41,7,1.0,39,0
...,...,...,...,...,...
478725,12962,20368,1.0,12579,22217
478726,12985,21058,1.0,12601,22218
478727,12989,22317,1.0,12605,22219
478728,13009,22339,1.0,12624,22220


In [13]:
def dataset_splits(interactions, num_users, num_items, validation_percentage: float, testing_percentage: float):
    seed = 1234

    (user_ids_training, user_ids_test,
     item_ids_training, item_ids_test,
     interactions_training, interactions_test) = train_test_split(interactions.mapped_user_id,
                                                        interactions.mapped_item_id,
                                                        interactions.Interaction,
                                                        test_size=testing_percentage,
                                                        shuffle=True,
                                                        random_state=seed)

    (user_ids_training, user_ids_validation,
     item_ids_training, item_ids_validation,
     interactions_training, interactions_validation) = train_test_split(user_ids_training,
                                                              item_ids_training,
                                                              interactions_training,
                                                              test_size=validation_percentage,
                                                             )

    urm_train = sp.csr_matrix((interactions_training, (user_ids_training, item_ids_training)),
                              shape=(num_users, num_items))

    urm_validation = sp.csr_matrix((interactions_validation, (user_ids_validation, item_ids_validation)),
                              shape=(num_users, num_items))

    urm_test = sp.csr_matrix((interactions_test, (user_ids_test, item_ids_test)),
                              shape=(num_users, num_items))



    return urm_train, urm_validation, urm_test



In [14]:
urm_train, urm_validation, urm_test = dataset_splits(interactions,
                                                     num_users=12638,
                                                     num_items=22222,
                                                     validation_percentage=0.20,
                                                     testing_percentage=0.20)

In [15]:
urm_train

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 306387 stored elements in Compressed Sparse Row format>

In [16]:
urm_validation

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 76597 stored elements in Compressed Sparse Row format>

In [17]:
urm_test

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>

#Build the Recommendation System

In [18]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2617 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2152 (17.0%) Users that have less than 1 test interactions


In [19]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame()

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

In [19]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
def objective_function_slim_bpr(optuna_trial):

    recommender_instance = RP3betaRecommender(urm_train)


    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 1, 100),
                  alpha = optuna_trial.suggest_float("alpha", 1e-2, 1, log=True),
                  beta = optuna_trial.suggest_float("beta", 1e-3, 1, log=True),
                  implicit = False,
                  normalize_similarity = True )

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [22]:
optuna_study = optuna.create_study(direction="maximize")

save_results = SaveResults()

optuna_study.optimize(objective_function_slim_bpr,
                      callbacks=[save_results],
                      n_trials = 200)

[I 2023-11-27 08:16:31,022] A new study created in memory with name: no-name-acc358fb-5299-41ab-a097-c277e3f7323b


RP3betaRecommender: URM Detected 466 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 322 ( 1.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2221.12 column/sec. Elapsed time 10.00 sec
EvaluatorHoldout: Processed 10039 (100.0%) in 6.02 sec. Users per second: 1668


[I 2023-11-27 08:16:47,876] Trial 0 finished with value: 0.025150156453324152 and parameters: {'topK': 15, 'alpha': 0.755195881724507, 'beta': 0.12199786123800652}. Best is trial 0 with value: 0.025150156453324152.


RP3betaRecommender: URM Detected 466 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 322 ( 1.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 1979.74 column/sec. Elapsed time 11.22 sec
EvaluatorHoldout: Processed 10039 (100.0%) in 6.61 sec. Users per second: 1518


[I 2023-11-27 08:17:07,867] Trial 1 finished with value: 0.028923393210922067 and parameters: {'topK': 81, 'alpha': 0.010600033373641933, 'beta': 0.04542667521682959}. Best is trial 1 with value: 0.028923393210922067.


...

[I 2023-11-27 09:18:49,135] Trial 197 finished with value: 0.029934327551122276 and parameters: {'topK': 40, 'alpha': 0.19734611062774954, 'beta': 0.15121471499414516}. Best is trial 195 with value: 0.030090634620219384.


RP3betaRecommender: URM Detected 466 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 322 ( 1.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2642.37 column/sec. Elapsed time 8.41 sec
EvaluatorHoldout: Processed 10039 (100.0%) in 8.33 sec. Users per second: 1205


[I 2023-11-27 09:19:07,517] Trial 198 finished with value: 0.029764731041003745 and parameters: {'topK': 39, 'alpha': 0.2294639804007496, 'beta': 0.2182220426334428}. Best is trial 195 with value: 0.030090634620219384.


RP3betaRecommender: URM Detected 466 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 322 ( 1.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2766.74 column/sec. Elapsed time 8.03 sec
EvaluatorHoldout: Processed 10039 (100.0%) in 7.99 sec. Users per second: 1257


[I 2023-11-27 09:19:24,796] Trial 199 finished with value: 0.029485431420634308 and parameters: {'topK': 42, 'alpha': 0.3455321269100255, 'beta': 0.05263021246527978}. Best is trial 195 with value: 0.030090634620219384.


In [23]:
bests = optuna_study.best_trial.params
bests

{'topK': 44, 'alpha': 0.21591232212788025, 'beta': 0.15299862700982697}

In [24]:
recommender_instance = RP3betaRecommender(urm_train + urm_validation)
recommender_instance.fit(**bests)

result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
result_df


RP3betaRecommender: URM Detected 230 ( 1.8%) users with no interactions.
RP3betaRecommender: URM Detected 93 ( 0.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 1554.33 column/sec. Elapsed time 14.30 sec
EvaluatorHoldout: Processed 10486 (100.0%) in 7.74 sec. Users per second: 1354


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.093143,0.162036,0.136137,0.048089,0.083478,0.267838,0.14892,0.110609,0.511158,0.354076,...,0.82972,0.424118,0.82972,0.086468,10.742446,0.998595,0.248248,0.827582,1.378839,0.299739


## Top popular for the users without info

In [25]:
class TopPopRecommender(object):

    def fit(self, URM_train):

        item_popularity = np.ediff1d(URM_train.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis = 0)


    def recommend(self, user_id, at=5):

        recommended_items = self.popular_items[0:at]

        return recommended_items

#Hyperparameter Tuning


*   Number of neighbors

*   Shrinking

*   Similarity Type



In [ ]:
%cd RecSys_Course_AT_PoliMi_
from Evaluation.Evaluator import EvaluatorHoldout


evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

[Errno 2] No such file or directory: 'RecSys_Course_AT_PoliMi_'
/gdrive/MyDrive/RECSYS/RecSys_Course_AT_PoliMi
EvaluatorHoldout: Ignoring 3066 (24.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2716 (21.5%) Users that have less than 1 test interactions


In [ ]:
def objective_function_ItemKNN_similarities(optuna_trial):

    recommender_instance = ItemKNNCFRecommender(urm_train)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])

    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 1000),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                  }

    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])


    recommender_instance.fit(**full_hyperp)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [ ]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

def objective_function_UserKNN_similarities(optuna_trial):

    recommender_instance = UserKNNCFRecommender(urm_train)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean', 'pearson'])

    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 1000),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                  }

    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])


    recommender_instance.fit(**full_hyperp)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [21]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame()

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

In [ ]:
item_optuna_study = optuna.create_study(direction="maximize")

item_save_results = SaveResults()

item_optuna_study.optimize(objective_function_ItemKNN_similarities,
                      callbacks=[item_save_results],
                      n_trials = 50)

In [ ]:
user_optuna_study = optuna.create_study(direction="maximize")

user_save_results = SaveResults()

user_optuna_study.optimize(objective_function_UserKNN_similarities,
                      callbacks=[user_save_results],
                      n_trials = 50)

In [ ]:
user_bests = user_optuna_study.best_trial.params

In [ ]:
item_bests = {'topK': 17, 'shrink': 158, 'tversky_alpha': 0.7658894054120393, 'tversky_beta': 17.560866787611264}

In [ ]:
recommender_instance_item = ItemKNNCFRecommender(urm_train + urm_validation)
recommender_instance_item.fit(**item_bests)

result_df_item, _ = evaluator_test.evaluateRecommender(recommender_instance_item)
result_df_item

ItemKNNCFRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 93 ( 0.4%) items with no interactions.
Similarity column 22222 (100.0%), 3253.92 column/sec. Elapsed time 6.83 sec
EvaluatorHoldout: Processed 10486 (100.0%) in 9.20 sec. Users per second: 1140


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.074661,0.134423,0.114083,0.038066,0.067712,0.229832,0.122869,0.090255,0.447358,0.292888,...,0.82972,0.371182,0.82972,0.032249,9.268157,0.99612,0.092586,0.714005,2.18609,0.272204


In [ ]:
recommender_instance_user = UserKNNCFRecommender(urm_train + urm_validation)
recommender_instance_user.fit(**user_optuna_study.best_trial.params)

result_df_user, _ = evaluator_test.evaluateRecommender(recommender_instance_user)
result_df_user

UserKNNCFRecommender: URM Detected 156 ( 1.2%) users with no interactions.
UserKNNCFRecommender: URM Detected 65 ( 0.3%) items with no interactions.
Similarity column 12638 (100.0%), 1790.04 column/sec. Elapsed time 7.06 sec
EvaluatorHoldout: Processed 9922 (100.0%) in 9.77 sec. Users per second: 1016


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.055614,0.119083,0.108848,0.026084,0.056232,0.185723,0.101881,0.073615,0.388228,0.219411,...,0.785093,0.304795,0.785093,0.012461,7.632904,0.983943,0.035661,0.587871,3.616883,0.23348


In [ ]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sp.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha = 0.5):
        self.alpha = alpha


    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [ ]:
recommender_instance_user = SLIM_BPR_Python(urm_train + urm_validation)
recommender_instance_user.fit(topK=100, epochs = 25, learning_rate=1e-8, lambda_i=5e-4)

recommender_instance_item = ItemKNNCFRecommender(urm_train + urm_validation)

item_bests = {'similarity': 'tversky',
 'topK': 18,
 'shrink': 10,
 'tversky_alpha': 0.19184100253588082,
 'tversky_beta': 1.6515107809400285}
recommender_instance_item.fit(**item_bests)


scoreshybridrecommender = ScoresHybridRecommender(urm_train, recommender_instance_item,recommender_instance_user)
scoreshybridrecommender.fit(alpha = 0.3)

result_df, _ = evaluator_test.evaluateRecommender(scoreshybridrecommender)
result_df

Recommender_Base_Class: URM Detected 230 ( 1.8%) users with no interactions.
Recommender_Base_Class: URM Detected 93 ( 0.4%) items with no interactions.
Epoch 1, Iteration 12638 in 6.34 seconds. Samples per second 1991.91
Epoch 2, Iteration 12638 in 3.10 seconds. Samples per second 4081.04
Epoch 3, Iteration 12638 in 3.35 seconds. Samples per second 3771.17
Epoch 4, Iteration 12638 in 3.86 seconds. Samples per second 3276.16
Epoch 5, Iteration 12638 in 2.86 seconds. Samples per second 4412.12
Epoch 6, Iteration 12638 in 2.72 seconds. Samples per second 4638.11
Epoch 7, Iteration 12638 in 2.64 seconds. Samples per second 4790.53
Epoch 8, Iteration 12638 in 2.44 seconds. Samples per second 5176.51
Epoch 9, Iteration 12638 in 2.61 seconds. Samples per second 4835.56
Epoch 10, Iteration 12638 in 3.20 seconds. Samples per second 3952.52
Epoch 11, Iteration 12638 in 4.09 seconds. Samples per second 3089.94
Epoch 12, Iteration 12638 in 1.74 seconds. Samples per second 7276.36
Epoch 13, Iterat

#Evaluate the Recommendation System

In [ ]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant) / relevant_items.shape[0]

    return recall_score


def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)

    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [ ]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = urm_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue

        recommendations = recommender.recommend(user_id=user_id,
                                               at=recommendation_length,
                                               urm_train=urm_train,
                                               remove_seen=True)

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemknn_recommender,
                                                                                            urm_train,
                                                                                            urm_test)

NameError: ignored

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

#Prepare Submission

In [26]:
#forse ho sbagliato ad agggiungere anche il test set qui
best_shrink = 5
urm_train_validation = urm_train + urm_validation + urm_test
urm_train_validation

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [28]:
best_recommender = RP3betaRecommender(urm_train_validation)
best_recommender.fit(**bests)

RP3betaRecommender: Similarity column 22222 (100.0%), 2276.31 column/sec. Elapsed time 9.76 sec


##Top popular for those users without info

In [29]:
toppop_recommender = TopPopRecommender()
toppop_recommender.fit(urm_train_validation)

In [30]:
users_to_recommend = np.array(data_target_users_test["user_id"])
len(users_to_recommend)

10882

In [31]:
def prepare_submission(interactions: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object, toppop_recommender: object):
    users_ids_and_mappings = interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates()
    items_ids_and_mappings = interactions[["item_id", "mapped_item_id"]].drop_duplicates()

    mapping_to_item_id = dict(zip(interactions.mapped_item_id, interactions.item_id))


    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id

        recommendations = recommender.recommend(user_id_array=mapped_user_id, cutoff=10)

        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    #assign top popular to the user without interactions
    users_without_info = np.setdiff1d(users_to_recommend,np.array(interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates().user_id))
    for user_id in users_without_info:
      recommendations = toppop_recommender.recommend(user_id, at = 10)
      submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    return submission

In [32]:
submission = prepare_submission(interactions, users_to_recommend, urm_train_validation, best_recommender, toppop_recommender)

In [33]:
submission

[(1, [101, 36, 403, 506, 123, 515, 139, 694, 254, 1546]),
 (2, [1095, 1522, 47, 28, 11, 196, 50, 102, 3176, 12]),
 (26, [139, 269, 254, 353, 314, 1089, 612, 165, 1005, 15]),
 (36, [4, 16, 181, 2205, 165, 379, 202, 314, 269, 283]),
 (41, [31, 58, 55, 592, 618, 77, 66, 605, 60, 148]),
 (47, [38, 214, 211, 96, 344, 262, 1635, 4, 442, 232]),
 (54, [19, 238, 2, 6, 8, 278, 174, 769, 112, 135]),
 (73, [125, 10057, 516, 998, 2105, 701, 21, 580, 1728, 786]),
 (88, [139, 346, 133, 11531, 1089, 1222, 99, 318, 77, 60]),
 (89, [138, 12673, 12672, 15014, 324, 15015, 344, 10, 154, 12677]),
 (95, [482, 2456, 173, 4, 44, 4627, 2178, 18638, 16, 669]),
 (100, [313, 764, 636, 99, 1050, 17, 262, 851, 3947, 112]),
 (101, [837, 82, 754, 2314, 624, 1290, 1177, 707, 526, 390]),
 (102, [2904, 2284, 2585, 7055, 2843, 5724, 2283, 3208, 2958, 67]),
 (104, [119, 299, 252, 23, 89, 2, 4, 129, 71, 639]),
 (114, [5, 17, 575, 5427, 50, 27, 4000, 7236, 42, 102]),
 (124, [715, 1342, 346, 859, 1198, 9878, 2590, 8112, 94, 2

In [34]:
def write_submission(submissions):
    with open("./submission.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [35]:
%cd /gdrive/MyDrive/RECSYS
write_submission(submission)

/gdrive/MyDrive/RECSYS
